In [0]:
%scala

case class DeviceIoTData (battery_level: Long, c02_level: Long, 
    cca2: String, cca3: String, cn: String, device_id: Long, 
    device_name: String, humidity: Long, ip: String, latitude: Double,
    lcd: String, longitude: Double, scale:String, temp: Long, timestamp: Long)

defined class DeviceIoTData

In [0]:
%scala

val ds = spark.read.json("/FileStore/iot_devices.json").as[DeviceIoTData]

ds.printSchema

display(ds.limit(5))

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122


In [0]:
%scala
// Detect failing devices with battery levels below a threshold.

ds.select($"battery_level", $"c02_level", $"device_name").where($"battery_level" < 8).sort($"c02_level")

display(ds.limit(5))

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122


In [0]:
%scala
//Identify offending countries with high levels of CO2 emissions.

val newDS = ds
  .filter(d => {d.c02_level > 1300})
  .groupBy($"cn")
  .avg()
  .sort($"avg(c02_level)".desc)

display(newDS.limit(10))


cn,avg(battery_level),avg(c02_level),avg(device_id),avg(humidity),avg(latitude),avg(longitude),avg(temp),avg(timestamp)
Solomon Islands,3.0,1588.0,187433.0,40.0,-9.43,159.95,21.0,1.458444060894E12
Federated States of Micronesia,3.0,1573.0,78806.0,55.0,6.92,158.25,13.0,1.45844405755E12
Rwanda,2.5,1560.5,102085.0,44.0,-2.0,30.0,21.5,1.458444058393E12
British Indian Ocean Territory,7.0,1560.0,83930.0,54.0,-6.0,71.5,27.0,1.458444057649E12
Aruba,4.0,1559.0,99899.0,84.0,12.51,-70.0,19.5,1.458444057927E12
Isle of Man,5.0,1548.0,137958.5,46.5,54.23,-4.57,24.5,1.4584440593755E12
Gambia,3.0,1544.5,34748.0,66.0,13.47,-16.57,16.0,1.458444056277E12
Lesotho,2.5,1537.5,74949.0,72.0,-29.41,27.990000000000002,21.0,1.458444057568E12
Cuba,5.2,1534.8,66819.2,42.0,21.444,-79.336,26.6,1.4584440572212E12
Gabon,8.0,1523.0,106953.0,30.0,0.38,9.45,28.0,1.45844405847E12


In [0]:
%scala
//Compute the min and max values for temperature, battery level, CO2, and humidity.

import org.apache.spark.sql.functions._ 

ds.select(min("temp"), max("temp"), min("humidity"), max("humidity"), min("c02_level"), max("c02_level"), min("battery_level"), max("battery_level"))

display(ds.limit(10))

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122
3,1210,US,USA,United States,6,sensor-pad-6al7RTAobR,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122
3,1129,CN,CHN,China,7,meter-gauge-7GeDoanM,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123
0,1536,JP,JPN,Japan,8,sensor-pad-8xUD6pzsQI,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123
3,807,JP,JPN,Japan,9,device-mac-9GcjZ2pw,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124
7,1470,US,USA,United States,10,sensor-pad-10BsywSYUF,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125


In [0]:
%scala
//Sort and group by average temperature, CO2, humidity, and country.

ds.filter(d => {d.temp > 25 && d.humidity > 75})
  .select("temp", "humidity", "cn")
  .groupBy($"cn")
  .avg()
  .sort($"avg(temp)".desc, $"avg(humidity)".desc).as("avg_humidity")

display(ds.limit(10))

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122
3,1210,US,USA,United States,6,sensor-pad-6al7RTAobR,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122
3,1129,CN,CHN,China,7,meter-gauge-7GeDoanM,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123
0,1536,JP,JPN,Japan,8,sensor-pad-8xUD6pzsQI,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123
3,807,JP,JPN,Japan,9,device-mac-9GcjZ2pw,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124
7,1470,US,USA,United States,10,sensor-pad-10BsywSYUF,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125
